In [1]:
import gensim

x_train = open('/content/drive/MyDrive/Colab Notebooks/film_plots_lemma_correct.txt',
               encoding='utf-8').readlines()

y_train = open('/content/drive/MyDrive/Colab Notebooks/wiki_titles.txt',
               encoding='utf-8').readlines()


x_test = open('/content/drive/MyDrive/Colab Notebooks/test_data_lemma_correct.txt',
              encoding='utf-8').readlines()

y_test = open('/content/drive/MyDrive/Colab Notebooks/test_titles.txt',
              encoding='utf-8').readlines()

print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))

model = open('/content/drive/MyDrive/Colab Notebooks/model.bin', 'rb')
model = gensim.models.KeyedVectors.load_word2vec_format(model, binary=True)

for idx, text in enumerate(x_train):
  x_train[idx] = [word for word in text.split() if word in model.vocab]

for idx, text in enumerate(x_test):
  x_test[idx] = [word for word in text.split() if word in model.vocab]

True
True


In [2]:
import pandas as pd


def ap(relev, k):    # average presicion
    ap = []
    for i in range(1, k):
        if relev[i-1] is 1:    # if doc is relevant
            ap.append(sum(relev[:i])/i)    # summary of precisions
    try: 
        ap = sum(ap)/sum(relev)
    except ZeroDivisionError:
        ap = 0.0
    return ap


def evaluation(query, relev, index=0, k=5):    # mean average precision (5)
    prec = round(sum(relev)/k, 4)    # rank is not take into account
    k += 1
    avp = ap(relev, k)    # rank is take into account
    evaluat = pd.DataFrame({'precision': prec,
                            'average_precision': avp},
                            index=[index])
    return evaluat


def calc_map(query, film, predictions, k, index):
    relev = [0] * k
    for i, pred in enumerate(predictions):
        if y_train[pred] == film:
            relev[i] = 1
    if relev != [0] * k:   
        df = evaluation(query, relev, index, k)
    else:
        df = pd.DataFrame({'precision': 0.0,
                           'average_precision': 0.0},
                           index=[index])
    return df


# map = sum(ap)/Q    # Q - number of quaries
# recall = sum(p)/Q

In [5]:
from annoy import AnnoyIndex # https://github.com/spotify/annoy
import numpy as np

f = 300
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for idx, text in enumerate(x_train):
  try:
    v = np.mean(model[text], axis=0)
  except ValueError:
    continue
  else:
    t.add_item(idx, v)

t.build(50) # 50 trees
t.save('test.ann')

True

In [6]:
u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, just mmap the file

df = pd.DataFrame(columns=['precision', 'average_precision'])

for idx, (query, film) in enumerate(zip(x_test, y_test)):
  v = np.mean(model[query], axis=0)
  top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

  df2 = calc_map(query, film, top_20, 20, idx)
  df = df.append(df2)


In [7]:
df.to_csv('map.csv')

MAP = pd.DataFrame(columns=['recall', 'MAP'])
df2 = pd.DataFrame({'recall': len([1 for prec in df.precision if prec != 0])/len(x_train),
                    'MAP': sum(df.average_precision)/len(x_train)},
                    index=[0])
MAP = MAP.append(df2)
MAP

,recall,MAP
0,0.001426,0.000704


In [11]:
# check if model works correctly

u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, just mmap the file

for idx, text in enumerate(x_train[:5]):
  v = np.mean(model[text], axis=0)
  top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

  print(top_20[0] == idx)

True
True
True
True
True
